In [ ]:
!pip install gym gym-retro
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib

Code for locating retro

In [ ]:
import os

import retro

retro_directory = os.path.dirname(retro.__file__)
game_dir = "data/stable/StreetFighterIISpecialChampionEdition-Genesis"
print(os.path.join(retro_directory, game_dir))

# Setup basic enviroment

In [ ]:
env.close()

Sample game

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
from utils.balanced_env import BalancedStreetFighterEnv

# Reset game to starting state
env = BalancedStreetFighterEnv()
obs = env.reset()

rewards = []

done = False
for game in range(1):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        rewards.append(reward)
        #time.sleep(0.01)
        print(info)

# Close the environment
env.close()

# Convert rewards to a numpy array for easier analysis
rewards = np.array(rewards)

# Identify points outside the range [-100, 100]
out_of_range_indices = np.where((rewards > 100) | (rewards < -100))[0]
out_of_range_rewards = rewards[out_of_range_indices]

# Plot rewards vs time
plt.figure(figsize=(10, 5))
plt.plot(rewards, label='Reward', color='blue')
plt.scatter(out_of_range_indices, out_of_range_rewards, color='red', label='Out of Range')
plt.axhline(0, color='red', linestyle='--', label='Zero Line')
plt.ylim(-100, 100)
plt.xlabel('Time Step')
plt.ylabel('Reward')
plt.title('Reward vs Time')
plt.legend()
plt.grid(True)
plt.show()

# Calculate and print statistics
mean_reward = np.mean(rewards)
std_reward = np.std(rewards)
print(f'Mean Reward: {mean_reward}')
print(f'Standard Deviation of Reward: {std_reward}')

# Plot histogram of rewards with out of range points
plt.figure(figsize=(10, 5))
plt.hist(rewards, bins=30, alpha=0.7, color='blue', edgecolor='black')
plt.axvline(mean_reward, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_reward:.2f}')
plt.xlabel('Reward')
plt.ylabel('Frequency')
plt.title('Histogram of Rewards')
plt.legend()
plt.grid(True)
plt.show()


## Optuna


In [ ]:
import optuna 
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.callbacks import CheckpointCallback
import os 

In [ ]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(OPT_DIR, exist_ok=True)

In [ ]:
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [ ]:
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [ ]:
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = StreetFighterEnv(style="balanced")
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        #model.learn(total_timesteps=300)
        model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        print(e)
        return -1000

## Crear estudio


In [ ]:
study_name = 'ppo_street_fighter'  # Nombre del estudio
storage_name = f'sqlite:///{OPT_DIR}/{study_name}.db'
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction='maximize')
study.optimize(optimize_agent, n_trials=100, n_jobs=1)

## Cargar estudio

In [ ]:
study_name = 'ppo_street_fighter'  # Nombre del estudio
storage_name = f'sqlite:///{OPT_DIR}/{study_name}.db'
study = optuna.load_study(study_name=study_name, storage=storage_name)


In [ ]:
# Obtener el mejor estudio
study.best_trial

In [ ]:
# Cargar el mejor estudio
model = PPO.load(os.path.join(OPT_DIR, 'trial_41_best_model.zip'))

# Guardar el mejor modelo

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Entrenar modelo

In [ ]:
# Create environment 

env = StreetFighterEnv(style="balanced")
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')


In [ ]:
model_params = study.best_params
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7
model_params

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

In [ ]:
model.load(os.path.join(CHECKPOINT_DIR, 'best_model_5000000.zip'))

In [ ]:
#model.learn(total_timesteps=1000, callback=callback)
model.learn(total_timesteps=5000000, callback=callback)

In [ ]:
model.save(os.path.join(log_dir, 'final_model'))

## Evaluar Modelo

In [ ]:
model_path = './logs/PPO_2'

# Cambiar los permisos del archivo
os.chmod(model_path, 0o777)

In [ ]:
model = PPO.load('./opt/trial_41_best_model.zip')

In [ ]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)


In [ ]:
mean_reward

# Testear modelo

In [ ]:
obs = env.reset()
obs.shape
env.step(model.predict(obs)[0])

In [ ]:
env.close()

In [ ]:
import time
import matplotlib.pyplot as plt
import cv2
total_rewards = []
victories = 0
defeats = 0
remaining_life_per_game = []
fourcc = cv2.VideoWriter_fourcc(*'XVID')
vw = cv2.VideoWriter('test2.avi', fourcc, 4, (320,224))
# Jugar 100 partidas
num_games = 3

for game in range(num_games):
    obs = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action = model.predict(obs)[0]
        env.render()
        obs, reward, done, info = env.step(action)
        #time.sleep(0.00001)
        vw.write(obs)
        total_reward += reward
    vw.release()
        
    total_rewards.append(total_reward)
    
    # Encontrar la información relevante en info (asumiendo que info es una lista de diccionarios)
    relevant_info = None
    for item in info:
        if 'health' in item:
            remaining_life_per_game.append(item['health'])
        if 'matches_won' in item and 'enemy_matches_won' in item:
            if item['matches_won'] > item['enemy_matches_won']:
                victories += 1
            else:
                defeats += 1
            break  # Salir del bucle una vez que se han contado las victorias y derrotas


In [ ]:
pip install imageio


In [ ]:
import time
import matplotlib.pyplot as plt

# Inicializar métricas
total_rewards = []
victories = 0
defeats = 0
remaining_life_per_game = []

# Jugar 100 partidas
num_games = 3
for game in range(num_games):
    # Resetear el entorno
    obs = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        # Predecir acción usando el modelo
        action = model.predict(obs)[0]
        # Tomar acción en el entorno
        env.render()
        time.sleep(0.001)
        obs, reward, done, info = env.step(action)
        # Acumular recompensa
        total_reward += reward
        
    # Guardar las métricas del juego
    total_rewards.append(total_reward)
    
    # Encontrar la información relevante en info (asumiendo que info es una lista de diccionarios)
    relevant_info = None
    for item in info:
        if 'health' in item:
            remaining_life_per_game.append(item['health'])
        if 'matches_won' in item and 'enemy_matches_won' in item:
            if item['matches_won'] > item['enemy_matches_won']:
                victories += 1
            else:
                defeats += 1
            break  # Salir del bucle una vez que se han contado las victorias y derrotas
    print(victories)

# Resultados finales
print("Total Rewards per Game:", total_rewards)
print("Total Victories:", victories)
print("Total Defeats:", defeats)
print("Remaining Life per Game:", remaining_life_per_game)

# Graficar los resultados
plt.figure(figsize=(15, 5))

# Gráfico de Recompensa Total por Juego
plt.subplot(1, 3, 1)
plt.plot(total_rewards, marker='o')
plt.title('Total Rewards per Game')
plt.xlabel('Game')
plt.ylabel('Total Reward')

# Gráfico de Vida Restante por Juego
plt.subplot(1, 3, 2)
plt.plot(remaining_life_per_game, marker='o', color='orange')
plt.title('Remaining Life per Game')
plt.xlabel('Game')
plt.ylabel('Remaining Life')

# Gráfico de Victorias y Derrotas
plt.subplot(1, 3, 3)
plt.bar(['Victories', 'Defeats'], [victories, defeats], color=['green', 'red'])
plt.title('Victories vs Defeats')
plt.ylabel('Count')

# Mostrar los gráficos
plt.tight_layout()
plt.show()


In [ ]:
backup_rewards = total_rewards 
backup_victories = victories
backup_defeats = defeats
backup_remaining_life_per_game = remaining_life_per_game

In [ ]:

# Resultados finales
print("Total Rewards per Game:", total_rewards)
print("Total Victories:", victories)
print("Total Defeats:", defeats)
print("Remaining Life per Game:", remaining_life_per_game)
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(total_rewards, marker='o')
plt.title('Total Rewards per Game')
plt.xlabel('Game')
plt.ylabel('Total Reward')
plt.savefig('total_rewards_per_game.png')

# Save the second plot as a PNG file
plt.subplot(1, 3, 2)
plt.plot(remaining_life_per_game, marker='o', color='orange')
plt.title('Remaining Life per Game')
plt.xlabel('Game')
plt.ylabel('Remaining Life')
plt.savefig('remaining_life_per_game.png')

# Save the third plot as a PNG file
plt.subplot(1, 3, 3)
plt.bar(['Victories', 'Defeats'], [victories, defeats], color=['green', 'red'])
plt.title('Victories vs Defeats')
plt.ylabel('Count')
plt.savefig('victories_vs_defeats.png')

# Close the figure to free up memory
